# Getting started with HuracanPy
In this notebook, we demonstrate the type of analysis workflow you can build using HuracanPy. To know more about the available options and functions, please go through the [huracanpy.load guide](index.rst), and/or browse the [API documentation](../api/index.rst). The [example gallery](../examples/index.rst) gallery also provides useful demonstrations.
We show three workflows: 
1. Studying a specific cyclone
2. Studying a set of tracks
3. Comparing a set of detected/modelled tracks to an observationnal reference

In [ ]:
import huracanpy

import matplotlib.pyplot as plt
import seaborn as sns

## 1. Studying a specific cyclone
In this example, we want to study hurricane Wilma (the deepest Atlantic hurricane on record).

### 1a. Load IBTrACS and subset the specific hurricane
Two subsets of IBTrACS are embedded within HuracanPy: WMO and JTWC. 
You can also retrieve the full and last IBTrACS file from the online website. 
Default behavior is loading the embedded WMO subset. For more information, see [huracanpy.load guide](load.ipynb).

In [ ]:
# Here we load the WMO subset. This raises a warning that reminds you of the main caveats. 
ib = huracanpy.load(source = "ibtracs")
## The tracks are loaded as an xarray.Dataset, with one dimension "record" corresponding to each point. 
## Variables indicate position in space and time, as well as additional attributes such as maximum wind speed and minimum slp.
ib 

In [ ]:
# Wilma corresponds to index 2005289N18282, so we subset this storm. There are two ways of doing this:
# 1. Use warray's where
Wilma = ib.where(ib.track_id == "2005289N18282", drop = True)
# 2. Use huracanpy's sel_id method (more efficient and shorter, but does the same thing)
# Note: the `.hrcn` is called an accessor, and allows you to call HuracanPy functions as methods on the xarray objects.
Wilma = ib.hrcn.sel_id("2005289N18282")
# The Wilma object contains only the data for Wilma:
Wilma

### 1b. Add category info
You can add the Saffir-Simpson and/or the pressure category of Wilma to the tracks (for full list of available info, see [huracanpy.info](../api/huracanpy.info.rst)).

In [ ]:
# Add Saffir-Simpson Category
Wilma = Wilma.hrcn.add_saffir_simpson_category(wind_name = "wind", wind_units = "knots")
Wilma.saffir_simpson_category # This is stored in the `saffir_simpson_category` variable

In [ ]:
# Add pressure category
Wilma = Wilma.hrcn.add_pressure_category(slp_name = "slp")
Wilma.pressure_category

In [ ]:
# Note: Most of the accessor methods have a get_* and an add_* version. 
# get_ return the values of what you ask for as a DataArray, while add_ adds it directly to the dataset with a default name. 
# In the previous case, we could have called get_pressure_category
Wilma.hrcn.get_pressure_category(slp_name = "slp")
# we could then save it as a variable, and potentially add it to the dataset separately

### 1c. Plot the track and its evolution

In [ ]:
# Plot the track on a map, colored by Saffir-Simpson category
Wilma.hrcn.plot_tracks(intensity_var_name = "saffir_simpson_category", scatter_kws = {"palette": "turbo"})

In [ ]:
# Plot intensity time series using matplotlib
fig, axs = plt.subplots(2, sharex = True)
axs[0].plot(Wilma.time, Wilma.wind)
axs[1].plot(Wilma.time, Wilma.slp)
axs[0].set_ylabel("Wind / kn")
axs[1].set_ylabel("SLP / hPa")

### 1d. Calculate properties 

#### Duration

In [ ]:
Wilma.hrcn.get_track_duration() # Note duration is in h

#### ACE

In [ ]:
## Compute ACE for each point
Wilma = Wilma.hrcn.add_ace(wind_units = "knots")
Wilma.ace

In [ ]:
## Plot cumulated ACE
plt.plot(Wilma.time, Wilma.ace.cumsum())

#### Translation speed

In [ ]:
# Compute translation speed
Wilma = Wilma.hrcn.add_translation_speed()
# Plot translation speed against latitude
plt.plot(Wilma.lat, Wilma.translation_speed)
plt.xlabel("Latitude / °")
plt.ylabel("Translation speed / m/s")

#### Intensification rate

In [ ]:
# Add intensification rate in wind and pressure
Wilma = Wilma.hrcn.add_rate(var_name = "wind")
Wilma = Wilma.hrcn.add_rate(var_name = "slp")
# NB: The rates will be in unit/s, where unit is the unit of the variable.

In [ ]:
# Plot intensity time series
fig, axs = plt.subplots(2, sharex = True)
axs[0].plot(Wilma.time, Wilma.rate_wind * 3600) # Convert to kn/h
axs[1].plot(Wilma.time, Wilma.rate_slp * 3600) # Convert to hPa/h
axs[0].set_ylabel("kn/h")
axs[1].set_ylabel("hPa/h")

## 2. Studying a set of tracks

### 2a. Loading data
Here we show an example with a csv file that is embedded within HuracanPy for example. HuracanPy supports many track files format, see [huracanpy.load guide](load.ipynb).

In [ ]:
# Loading the ERA5 1996 TC tracks
## The tracks detected by TempestExtremes in ERA5 for the year 1996 are embedded within the package as an example.
file = huracanpy.example_year_file
print(file)
## Load the tracks with huracanpy.load. 
## Here the file extension is '.csv', the function will automatically recognise how to open it.
tracks = huracanpy.load(file)
## The tracks are loaded as an xarray.Dataset, with one dimension "record" corresponding to each point. 
## Variables indicate position in space and time, as well as additional attributes such as maximum wind speed and minimum slp.
tracks

### 2b. Adding info to the tracks
HuracanPy has several function to add useful information to the tracks (for full list, see [huracanpy.info](../api/huracanpy.info.rst)). Here for example we add basin and SSHS category information.

In [ ]:
# Add basin
tracks = tracks.hrcn.add_basin() # Add basin attribute
tracks.basin

In [ ]:
# Show distribution of TC points among basins (calling seaborn function, works better with categorical labels)
import seaborn as sns
sns.countplot(tracks.basin) 

In [ ]:
# Add SSHS and pressure categories
tracks = tracks.hrcn.add_saffir_simpson_category(wind_name = "wind10", wind_units='m s-1') 
tracks = tracks.hrcn.add_pressure_category(slp_name = "slp",) 
## (In ERA5 data, wind is stored in wind10 in m/s)
tracks[["saffir_simpson_category", "pressure_category"]]

In [ ]:
# Show distribution of TC points among categories (using xarray's built-in function)
tracks.saffir_simpson_category.plot.hist(bins = [-1.5, -0.5,  0.5,  1.5,  2.5,  3.5,  4.5,  5.5], alpha = 0.5)
tracks.pressure_category.plot.hist(bins = [-1.5, -0.5,  0.5,  1.5,  2.5,  3.5,  4.5,  5.5], alpha = 0.5)

### 2c. Plotting
HuracanPy embeds basic plotting functions, which are mainly meant for having a preliminary look at your data. In particular here we show how to plot the track points themselves, and track density. You can learn more in the [huracanpy.plot guide](plot.ipynb). The [example gallery](../examples/index.rst)  also displays nice plots made from HuracanPy and the associated scripts.
#### Plotting the tracks

In [ ]:
# Plot ERA5 tracks colored by wind intensity
tracks.hrcn.plot_tracks(intensity_var_name = "wind10",)

#### Plotting track density

In [ ]:
# You can plot the track density directly with `plot_density`, which is based on a simple 2D histogram of TC points
tracks.hrcn.plot_density()

In [ ]:
# You can also get the underlying density matrix with `get_density` and then use it to make you own plots in your favourite way
tracks.hrcn.get_density()

#### Plotting genesis points

In [ ]:
# `get_gen_vals` allows you to subset only the genesis points in an efficient way
gen_points = tracks.hrcn.get_gen_vals()
gen_points

In [ ]:
# If you use `plot_tracks` on these, you can display only the genesis points.
gen_points.hrcn.plot_tracks()

### 2d. Compute statistics

#### Number of cyclones

In [ ]:
tracks.track_id.hrcn.nunique() # Count number of unique track ids

#### Cyclones duration & TC days

In [ ]:
## Get the duration for each track
TC_duration = tracks.hrcn.get_track_duration() 
TC_duration # xarray.Dataset with track_id as dimension

In [ ]:
## Compute the total number of TC days
## Sum all the durations (and divide by 24 because durations are in hours)
TC_duration.sum() / 24

#### Cyclone Intensity

In [ ]:
# There are two ways to obtain the lifetime maximum intensity (LMI) of each tracks
## 1. Use `get_apex_vals`, which return the subset of points only as specified LMI
tracks.hrcn.get_apex_vals(varname = "wind10")

In [ ]:
## 2. Compute lifetime maximum intensity per track with xarray's groupby
LMI_wind = tracks.groupby("track_id").max().wind10
LMI_wind # xarray.Dataset with track_id as dimension

In [ ]:
# You can then plot the LMI distribution using xarray's built-in plot function.
LMI_wind.plot.hist()

#### ACE

In [ ]:
# Compute ACE for each point
tracks = tracks.hrcn.add_ace(wind_name = "wind10", wind_units = 'm s**-1')
tracks.ace

In [ ]:
## Compute total ACE
tracks.ace.sum()

### 2e. Compositing lifecycle

In [ ]:
# Add time from apex variable to be able to superimpose all the tracks centered on apex
tracks = tracks.hrcn.add_time_from_apex(intensity_var_name='slp', stat = "min") # Add time from minimum pressure
tracks.time_from_apex

In [ ]:
# Plot composite SLP lifecycle
## Convert time_from_apex to hours
tracks["time_from_apex"] = tracks.time_from_apex * 1e-9 / 3600
## Use xarray's where to mask points too far away from apex (100 hours away)
tracks_close_to_apex = tracks.where((tracks.time_from_apex >= -100) & (tracks.time_from_apex <= 100), drop = True)
## Seaborn lineplot allows for drawing composites with uncertainty range
sns.lineplot(x = tracks_close_to_apex.time_from_apex, # x-axis is time from apex
             y = tracks_close_to_apex.slp / 100,) # y-axis is slp, converted to hPa

## 3. Comparing two datasets
In this part, we compare the set of 1996 tracks above to IBTrACS which we use as reference. 
To start with, note that for all that was shown above, you can superimpose several sets and therefore compare several sources/models/trackers/etc. Below we show specific functions for matching tracks and computing detection scores.

### 3a. Data

In [ ]:
# Remember IBTrACS is stored in the `ib` object from the first part above. 
# Here we subset the 1996 tracks with xarray's where method: 
ib_1996 = ib.where(ib.time.dt.year == 1996, drop = True)
ib_1996

In [ ]:
# The tracks from ERA5 are stored in `tracks`. For clarity, we name it `ERA5` from now:
ERA5 = tracks.copy()

### 3b. Superimposing several sets on one plot
To start with, note that for all that was shown above, you can superimpose several sets and therefore compare several sources/models/trackers/etc. Here we only show one example.

In [ ]:
# Compute LMI for both sets
LMI_wind_ib = ib_1996.groupby("track_id").max().wind
LMI_wind_ib = LMI_wind_ib / 1.94 # Convert kn to m/s
LMI_wind_ERA5 = ERA5.groupby("track_id").max().wind10
# Plot both histograms
LMI_wind_ib.plot.hist(bins = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65], color = 'k', label = "IBTrACS")
LMI_wind_ERA5.plot.hist(bins = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65], label = "ERA5", alpha = 0.8)
# Labels
plt.legend()
plt.xlabel("Lifetime maximum wind speed / m/s")
plt.ylabel("Number of tracks")

### 3c. Matching tracks

In [ ]:
matches = huracanpy.assess.match([ERA5, ib_1996], names = ["ERA5", "IBTrACS"])
matches # each row is a pair of tracks that matched, with both ids, the number of time steps and the mean distance between the tracks over their matching period.

### 3d. Computing scores

In [ ]:
# Probability of detection (POD) : Proportion of observed tracks that are found in ERA5.
huracanpy.assess.pod(matches, ref = ib_1996, ref_name = "IBTrACS")

In [ ]:
# False alarm rate (FAR) : Proportion of detected tracks that were not observed
huracanpy.assess.far(matches, detected = ERA5, detected_name = "ERA5")

### 3e. Venn diagrams
Venn diagrams are a convenient way to show the overlap between two datasets.

In [ ]:
huracanpy.plot.venn([ERA5, ib_1996], matches, labels = ["ERA5", "IBTrACS"])